# Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup # package to transform the data of webpage into pandas dataframe
import requests # library to handle requests

# Retreive Wikipedia data and place into the Dataframe

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = BeautifulSoup(requests.get(url).content, 'lxml')

In [3]:
# obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
table = content.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())
    
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood        
wiki_df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']

# Ignore cells with a borough that is Not assigned.
wiki_df['Borough'].replace('Not assigned', np.nan, inplace=True)
wiki_df.dropna(subset=['Borough'], inplace=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
wiki_df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

# More than one neighborhood can exist in one postal code area. 
# These two rows will be combined into one row with the neighborhoods separated with a comma.
wiki_df = wiki_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
wiki_df.columns = ['Postal Code', 'Borough', 'Neighborhood']
wiki_df.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
wiki_df.shape

(103, 3)

In [5]:
url="http://cocl.us/Geospatial_data"
geospatial_df = pd.read_csv(url)
geospatial_df.columns = ['Postal Code', 'Latitude', 'Longitude']
toronto_metro_df = pd.merge(wiki_df, geospatial_df, on=['Postal Code'], how='inner')
toronto_metro_df .head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_metro_df ['Borough'].unique()),
        toronto_metro_df .shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.
